In [1]:
import pandas as pd
from data.OpenL3_embeddings import EmbeddingsOpenL3
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from essentia.standard import MonoLoader, TensorflowPredict2D, TensorflowPredictEffnetDiscogs

from main.choices import genre_weightage, tags_weightage, instrument_weightage, feature_weights, mdata_outliers, normalizing_factor, balancing_factor, normalizing_factor_meta
from pipeline_code import embedding_model_weights_dsg, embedding_model_weights_l3, all_columns_to_drop, rename_labels
from unidecode import unidecode
from main import choices
from sklearn.feature_extraction.text import TfidfVectorizer
import json
import numpy as np
import music_tag
import regex as re

2023-06-10 00:40:44.635465: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-10 00:40:44.635493: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[   INFO   ] MusicExtractorSVM: no classifier models were configured by default
2023-06-10 00:40:45.075810: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-10 00:40:45.076564: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: canno

No new songs to extract


In [2]:
path = "data/all_classifiers_and_metadata/"
song_path = 'media/'

In [3]:
def extract_all_features(all_songs, df_extract=None, single_file=None):
    column_labels = ['song name']
    rows = []
    flag = True
    with open('data/weights_metadata.json') as json_file:
        model_weights_metadata = json.load(json_file)
    json_file.close()

    for song in all_songs:
        extractor = EmbeddingsOpenL3(embedding_model_weights_l3)
        embeddings_l3 = extractor.compute(song_path + song)

        audio = MonoLoader(filename=song_path+song, sampleRate=44100, resampleQuality=4)()
        embedding_model = TensorflowPredictEffnetDiscogs(graphFilename=embedding_model_weights_dsg, output="PartitionedCall:1")
        embeddings_dsg = embedding_model(audio)

        classification_models = {}
        for key in model_weights_metadata:
            model_type = model_weights_metadata[key][0]
            file_name = model_weights_metadata[key][1]

            weight_file = path + file_name + ".pb"
            mdata_file = path + file_name + ".json"

            metadata = json.load(open(mdata_file, 'r'))
            input_ = metadata['schema']['inputs'][0]['name']
            output = metadata['schema']['outputs'][0]['name']
            classes = metadata['classes']

            model = TensorflowPredict2D(graphFilename=weight_file, output=output, input=input_)
            
            classification_models[key] = [model, model_type, classes]
            
        scores = [song] if single_file else [df_extract[df_extract['file_path'] == song]['song name'].values[0]]

        for key in classification_models:
            model = classification_models[key][0]
            model_type = classification_models[key][1]
            classes = classification_models[key][2]

            if model_type == "openl3":
                predictions = np.mean(model(embeddings_l3), axis=0)
            else:
                predictions = np.mean(model(embeddings_dsg), axis=0)

            if flag:
                for i in range(0, len(predictions)):
                    label = str(key) + "_" + str(classes[i])
                    column_labels.append(label)
        
            for i in range(0, len(predictions)):
                scores.append(predictions[i])
                
        flag = False
        rows.append(scores)
    return column_labels, rows


In [14]:
df = pd.read_csv('data/songs_db.csv')
df_features = pd.read_csv('data/song_dataset_final.csv')
meta_df = pd.read_csv('data/metadata.csv')
file_path = 'uploads/temp_neeraj.m4a'

In [5]:
column_labels, rows = extract_all_features([file_path], single_file=file_path)

[   INFO   ] TensorflowPredict: Successfully loaded graph file: `data/all_classifiers_and_metadata/openl3-music-mel128-emb512-3.pb`
2023-06-10 00:40:51.208047: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2095965000 Hz
2023-06-10 00:40:51.495872: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 391249920 exceeds 10% of free system memory.
2023-06-10 00:40:51.741712: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 391249920 exceeds 10% of free system memory.
2023-06-10 00:40:52.129860: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 401356800 exceeds 10% of free system memory.
2023-06-10 00:40:52.273022: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 401356800 exceeds 10% of free system memory.
2023-06-10 00:40:52.435766: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 391249920 exceeds 10% of free system memory.
[   INFO   ] TensorflowPredict: Successfully lo

In [15]:
new_df = pd.DataFrame(rows, columns=column_labels)
new_df

,song name,danceability_danceable,danceability_not_danceable,mood_acoustic_acoustic,mood_acoustic_non_acoustic,mood_aggressive_aggressive,mood_aggressive_not_aggressive,mood_electronic_electronic,mood_electronic_non_electronic,mood_happy_happy,...,Tag_piano,Tag_strings,Tag_synthesizer,Tag_violin,Tag_voice,Tag_emotional,Tag_energetic,Tag_film,Tag_happy,Tag_relaxing
0,uploads/temp_neeraj.m4a,0.342031,0.657969,0.033019,0.966981,0.602906,0.397094,0.481588,0.518411,0.044799,...,0.03164,0.014006,0.245114,0.009679,0.007887,0.012963,0.005592,0.034947,0.004477,0.021609


In [16]:
new_df.drop(columns=all_columns_to_drop, inplace=True)
new_df.rename(columns=rename_labels, inplace=True)

genre_columns = [col for col in new_df.columns if col.startswith("Genre")]
tag_columns = [col for col in new_df.columns if col.startswith("Tag")]

genre_data = new_df[genre_columns].T
tag_data = new_df[tag_columns].T

top_genres, top_tags = [], []
for col in genre_data.columns:
    temp1 = genre_data[col].nlargest(3).index.to_list()
    temp2 = tag_data[col].nlargest(5).index.to_list()
    top_genres.append(temp1)
    top_tags.append(temp2)

new_df['top_genres'] = top_genres
new_df['top_tags'] = top_tags

updated_dataframe = pd.concat([df_features, new_df], ignore_index=True)

In [17]:
temp = []
audio_file = music_tag.load_file(song_path + file_path)

temp.append(file_path)
temp.append(str(audio_file['artist']))
temp.append(str(audio_file['album']))
temp.append(int(audio_file['tracknumber']))
temp.append(str(audio_file['albumartist']))
temp.append(int(audio_file['year']))

temp_df = pd.DataFrame([temp], columns=meta_df.columns)
meta_df = pd.concat([meta_df, temp_df], ignore_index=True)

In [18]:
df = pd.concat([df, pd.DataFrame([[df['ID'].max() + 500, file_path, None]], columns=df.columns)], ignore_index=True)
df['year'] = meta_df['year']
merged_df = pd.merge(df, updated_dataframe, on='song name', how='outer')

In [20]:
genre_columns = [col for col in merged_df.columns if col.startswith("Genre")]
tag_columns = [col for col in merged_df.columns if col.startswith("Tag")]
instrument_columns = [col for col in merged_df.columns if col.startswith("Instrument")]

In [21]:
X = merged_df.iloc[:, 3:-2].copy()
X['year'] = X['year'].replace(0, np.nan).fillna(X['year'].median())
minmax = MinMaxScaler(feature_range=(0,1))
X.iloc[:,:14] = minmax.fit_transform(X.iloc[:,:14])

X['voice_male'] = (1 - X['voice_female']) * X['overall_voice']
X['voice_female'] = X['voice_female'] * X['overall_voice']

instrument_weightage = choices.instrument_weightage
feature_weights = choices.feature_weights
balancing_factor = choices.balancing_factor

In [22]:
for key in feature_weights:
    X[key] = X[key] * feature_weights[key]
for col in genre_columns:
    X[col] = X[col] * genre_weightage
for col in instrument_columns:
    X[col] = X[col] * instrument_weightage
for col in tag_columns:
    X[col] = X[col] * tags_weightage

df_cosine = pd.DataFrame(cosine_similarity(X, dense_output=True))
df_cosine = df_cosine.applymap(lambda x: np.power(x, normalizing_factor))
indices = pd.Series(merged_df.index, index = merged_df['song name'])

In [23]:
def remove_punct(text):
    return unidecode(re.sub(r'[^\w\s\,]', '', text.lower())) if str(text) != 'nan' else ''

def remove_outliers_and_extra_space(text):
    for substring in mdata_outliers:
        text = text.replace(substring, '')
    clean_text = re.sub(r'\s+', ' ', text).strip()
    return clean_text

artists = meta_df['artist'].apply(lambda x: remove_punct(x).split(', '))
albums = meta_df['album'].apply(lambda x: remove_punct(x).split(', '))
album_artist = meta_df['album artist'].apply(lambda x: remove_punct(x).split(', '))

meta_df['artists_album'] = artists + album_artist + albums
meta_df['artists_album'] = meta_df['artists_album'].apply(lambda x: [remove_outliers_and_extra_space(i) for i in x])
meta_df['artists_album'] = meta_df['artists_album'].apply(lambda x: list(set(x)))
meta_df['artists_album_final'] = meta_df['artists_album'].apply(lambda x: " ".join([text.replace(" ", "_") for text in x]))

In [24]:
tfidf = TfidfVectorizer(stop_words = "english")
tfidf_matrix = tfidf.fit_transform(meta_df['artists_album_final'])

In [27]:
df_cosine_meta = pd.DataFrame(cosine_similarity(tfidf_matrix, tfidf_matrix))
df_cosine_meta = df_cosine_meta.applymap(lambda x: np.power(x, normalizing_factor_meta) * balancing_factor)
resultant_cosine = df_cosine.add(df_cosine_meta)

In [37]:
index = indices[file_path]
similarity_scores = list(enumerate(resultant_cosine[index]))
similarity_scores = sorted(similarity_scores, key = lambda x: x[1],reverse = True)
similarity_scores = similarity_scores[1:16]
res_indices = [i[0] for i in similarity_scores]

In [38]:
similars = merged_df['ID'].iloc[res_indices[:]].to_list()

In [39]:
df['sort'] = pd.Categorical(df['ID'], categories=similars, ordered=True)
df_sorted = df.sort_values('sort')
df_sorted[:15]

,ID,song name,file_path,year,sort
134,1777,Ready Steady Go (Korean Style) - Paul Oakenfold,audio/Ready_Steady_Go_Korean_Style_-_Paul_Oake...,2002,1777
140,1875,Sadda Dil Vi Tu (Ga Ga Ga Ganpati) - Sachin-Ji...,audio/Sadda_Dil_Vi_Tu_Ga_Ga_Ga_Ganpati_-_Sachi...,2013,1875
1,1892,"Aaluma Doluma - Anirudh Ravichander, Badshah",audio/Aaluma_Doluma_-_Anirudh_Ravichander_Bads...,2015,1892
105,1805,"Naatu Naatu - Rahul Sipligunj, Kaala Bhairava,...",audio/Naatu_Naatu_-_Rahul_Sipligunj_Kaala_Bhai...,2021,1805
163,1877,Thunder - Imagine Dragons,audio/Thunder_-_Imagine_Dragons.m4a,2017,1877
176,1865,"Vaathi Coming - Anirudh Ravichander, Gana Bala...",audio/Vaathi_Coming_-_Anirudh_Ravichander_Gana...,2020,1865
179,1738,Warrior - Aurora,audio/Warrior_-_Aurora.m4a,2020,1738
68,1749,Invisible - Julius Dreisig & Zeus X Crona,audio/Invisible_-_Julius_Dreisig__Zeus_X_Crona...,2018,1749
133,1905,Radioactive - Imagine Dragons,audio/Radioactive_-_Imagine_Dragons.m4a,2012,1905
61,1895,I Don't Do Personals - Ramin Djawadi,audio/I_Dont_Do_Personals_-_Ramin_Djawadi.m4a,2023,1895


In [ ]:
def get_similar_songs_id(request, id):
    df = pd.read_csv('data/songs_db.csv')
    df_features = pd.read_csv('data/song_dataset_final.csv')
    meta_df = pd.read_csv('data/metadata.csv')
    df['year'] = meta_df['year']
    merged_df = pd.merge(df, df_features, on='song name', how='inner')
    
    genre_columns = [col for col in merged_df.columns if col.startswith("Genre")]
    tag_columns = [col for col in merged_df.columns if col.startswith("Tag")]
    instrument_columns = [col for col in merged_df.columns if col.startswith("Instrument")]
    
    X = merged_df.iloc[:, 3:-2].copy()
    X['year'] = X['year'].replace(0, np.nan).fillna(X['year'].median())
    minmax = MinMaxScaler(feature_range=(0,1))
    X.iloc[:,:14] = minmax.fit_transform(X.iloc[:,:14])

    X['voice_male'] = (1 - X['voice_female']) * X['overall_voice']
    X['voice_female'] = X['voice_female'] * X['overall_voice']

    instrument_weightage = choices.instrument_weightage
    feature_weights = choices.feature_weights
    balancing_factor = choices.balancing_factor

    if request.user.is_authenticated:
        pref = request.user.preference
        if pref:
            feature_weights['Engagement'] = pref.engagement
            feature_weights['danceability'] = pref.danceability
            feature_weights['mood_acoustic'] = pref.acoustics
            feature_weights['mood_aggressive'] = pref.aggressive
            feature_weights['mood_happy'] = pref.happy
            feature_weights['mood_party'] = pref.party
            feature_weights['mood_relaxed'] = pref.relaxed
            feature_weights['mood_sad'] = pref.sad
            feature_weights['tonal'] = pref.tonality
            feature_weights['Reverb_wet'] = pref.reverb
            feature_weights['voice_female'] = pref.gender
            feature_weights['voice_male'] = pref.gender
            feature_weights['overall_voice'] = pref.voice
            feature_weights['year'] = pref.year
            instrument_weightage = pref.instrument
            balancing_factor = pref.metadata

    for key in feature_weights:
        X[key] = X[key] * feature_weights[key]
    for col in genre_columns:
        X[col] = X[col] * genre_weightage
    for col in instrument_columns:
        X[col] = X[col] * instrument_weightage
    for col in tag_columns:
        X[col] = X[col] * tags_weightage

    df_cosine = pd.DataFrame(cosine_similarity(X, dense_output=True))
    df_cosine = df_cosine.applymap(lambda x: np.power(x, normalizing_factor))
    indices = pd.Series(merged_df.index, index = merged_df['ID'])

    def remove_punct(text):
        return unidecode(re.sub(r'[^\w\s\,]', '', text.lower())) if str(text) != 'nan' else ''

    def remove_outliers_and_extra_space(text):
        for substring in mdata_outliers:
            text = text.replace(substring, '')
        clean_text = re.sub(r'\s+', ' ', text).strip()
        return clean_text
    
    artists = meta_df['artist'].apply(lambda x: remove_punct(x).split(', '))
    albums = meta_df['album'].apply(lambda x: remove_punct(x).split(', '))
    album_artist = meta_df['album artist'].apply(lambda x: remove_punct(x).split(', '))
    
    meta_df['artists_album'] = artists + album_artist + albums
    meta_df['artists_album'] = meta_df['artists_album'].apply(lambda x: [remove_outliers_and_extra_space(i) for i in x])
    meta_df['artists_album'] = meta_df['artists_album'].apply(lambda x: list(set(x)))
    meta_df['artists_album_final'] = meta_df['artists_album'].apply(lambda x: " ".join([text.replace(" ", "_") for text in x]))

    tfidf = TfidfVectorizer(stop_words = "english")
    tfidf_matrix = tfidf.fit_transform(meta_df['artists_album_final'])

    df_cosine_meta = pd.DataFrame(cosine_similarity(tfidf_matrix, tfidf_matrix))
    df_cosine_meta = df_cosine_meta.applymap(lambda x: np.power(x, normalizing_factor_meta) * balancing_factor)
    resultant_cosine = df_cosine.add(df_cosine_meta)

    index = indices[id]
    similarity_scores = list(enumerate(resultant_cosine[index]))
    similarity_scores = sorted(similarity_scores, key = lambda x: x[1],reverse = True)
    similarity_scores = similarity_scores[1:16]
    res_indices = [i[0] for i in similarity_scores]
    return merged_df['ID'].iloc[res_indices[:]].to_list()

